In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shuttledat/shuttle (1).trn
/kaggle/input/shuttledat/shuttle (1).tst


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.utils import compute_class_weight
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [3]:
dtrn = pd.read_csv("/kaggle/input/shuttledat/shuttle (1).trn", sep=",", header='infer' )
dtes = pd.read_csv("/kaggle/input/shuttledat/shuttle (1).tst", sep=",", header='infer' )

In [4]:
dtr=dtrn.to_numpy()
dte=dtes.to_numpy()

In [5]:
dtr.shape

(43499, 1)

In [6]:
dte.shape

(14499, 1)

In [7]:
ln=[]
yn=[]
for i in range(43499):
    l=list(map(int,dtr[i][0].split()))
    ln.append(l[:9])
    yn.append(l[9])

In [8]:
lnn=[]
ynn=[]
for i in range(14499):
    l=list(map(int,dte[i][0].split()))
    lnn.append(l[:9])
    ynn.append(l[9])

In [9]:
yok=np.array(yn)
ysi=pd.Series(yok) 
ysi.value_counts()

1    34108
4     6748
5     2458
3      132
2       36
7       11
6        6
dtype: int64

In [10]:
yok=np.array(ynn)
ysi=pd.Series(yok) 
ysi.value_counts()

1    11478
4     2154
5      809
3       39
2       13
6        4
7        2
dtype: int64

TREATING CLASS 3 AS A MINORITY:

In [11]:
X = np.asarray(ln, dtype=np.float32)
Xtes = np.asarray(lnn, dtype=np.float32)

In [12]:
yk=[]
for i in yn:
    if i==3:
        yk.append(1)
    else:
        yk.append(0)

In [13]:
ykk=[]
for i in ynn:
    if i==3:
        ykk.append(1)
    else:
        ykk.append(0)

In [14]:
ytrain=np.array(yk)
ytest=np.array(ykk)

In [15]:
print("Before OverSampling, counts of label '1': {}".format(sum(ytrain==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(ytrain==0)))
from imblearn.over_sampling import SMOTE
X_train_res,y_train_res = SMOTE().fit_resample(X, ytrain)

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

Before OverSampling, counts of label '1': 132
Before OverSampling, counts of label '0': 43367 

After OverSampling, the shape of train_X: (86734, 9)
After OverSampling, the shape of train_y: (86734,) 

After OverSampling, counts of label '1': 43367
After OverSampling, counts of label '0': 43367


In [16]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
from sklearn.metrics import f1_score
from statistics import stdev

2.4.1


NEURAL NETWORK MODEL:

In [17]:
def callf1(xx,yy,xt,yt):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    
    model.compile(optimizer='adam',
                  loss='mean_absolute_error',
                  metrics=['accuracy'])
    model.fit(xx, yy , epochs=10)
    ls=[]
    test_loss, test_acc = model.evaluate(xt,  yt, verbose=2)
    print('\nTest accuracy:', test_acc)
    tr_loss, tr_acc = model.evaluate(xx,  yy, verbose=2)
    ls.append(test_acc)
    ls.append(tr_acc)
    ypr=model.predict(xt)
    ypr=(ypr>0.5)*1
    ypre= np.ravel(ypr)
    ls.append(f1_score(yt, ypre))
    return ls

In [21]:
sta=[]
ste=[]
sf =[]
for i in range(30):
    r=callf1(X, ytrain.ravel(),Xtes,ytest.ravel())
    ste.append(r[0])
    sta.append(r[1])
    sf.append(r[2])

Epoch 1/10
1360/1360 [==============================] - 3s 2ms/step - loss: 1.5044 - accuracy: 0.7920
Epoch 2/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.3284 - accuracy: 0.9853
Epoch 3/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.1668 - accuracy: 0.9959
Epoch 4/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.1114 - accuracy: 0.9957
Epoch 5/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0558 - accuracy: 0.9953
Epoch 6/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0313 - accuracy: 0.9961
Epoch 7/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0235 - accuracy: 0.9958
Epoch 8/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0086 - accuracy: 0.9964
Epoch 9/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0042 - accuracy: 0.9969
Epoch 10/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.003

1360/1360 [==============================] - 3s 2ms/step - loss: 0.5080 - accuracy: 0.9758
Epoch 3/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.1904 - accuracy: 0.9956
Epoch 4/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0915 - accuracy: 0.9955
Epoch 5/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0405 - accuracy: 0.9959
Epoch 6/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0177 - accuracy: 0.9961
Epoch 7/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0111 - accuracy: 0.9960
Epoch 8/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0036 - accuracy: 0.9970
Epoch 9/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0033 - accuracy: 0.9968
Epoch 10/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0033 - accuracy: 0.9969
454/454 - 1s - loss: 0.0028 - accuracy: 0.9973

Test accuracy: 0.9973101615905762
1360/1360 

1360/1360 [==============================] - 3s 2ms/step - loss: 0.1940 - accuracy: 0.9956
Epoch 4/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0888 - accuracy: 0.9959
Epoch 5/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0707 - accuracy: 0.9951
Epoch 6/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0342 - accuracy: 0.9951
Epoch 7/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0173 - accuracy: 0.9967
Epoch 8/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0110 - accuracy: 0.9979
Epoch 9/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0086 - accuracy: 0.9980
Epoch 10/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0058 - accuracy: 0.9988
454/454 - 1s - loss: 0.0013 - accuracy: 0.9992

Test accuracy: 0.9991723299026489
1360/1360 - 1s - loss: 0.0012 - accuracy: 0.9994
Epoch 1/10
1360/1360 [==============================] - 3s 2ms/

1360/1360 [==============================] - 3s 2ms/step - loss: 0.0911 - accuracy: 0.9956
Epoch 5/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0339 - accuracy: 0.9963
Epoch 6/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0121 - accuracy: 0.9962
Epoch 7/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0043 - accuracy: 0.9973
Epoch 8/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0023 - accuracy: 0.9985
Epoch 9/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0015 - accuracy: 0.9989
Epoch 10/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0015 - accuracy: 0.9990
454/454 - 1s - loss: 0.0012 - accuracy: 0.9994

Test accuracy: 0.9993792772293091
1360/1360 - 1s - loss: 0.0012 - accuracy: 0.9992
Epoch 1/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.8606 - accuracy: 0.8983
Epoch 2/10
1360/1360 [==============================] - 3s 2ms/

1360/1360 [==============================] - 3s 2ms/step - loss: 0.0822 - accuracy: 0.9956
Epoch 6/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0251 - accuracy: 0.9962
Epoch 7/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0136 - accuracy: 0.9961
Epoch 8/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0079 - accuracy: 0.9966
Epoch 9/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0045 - accuracy: 0.9970
Epoch 10/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.0038 - accuracy: 0.9970
454/454 - 1s - loss: 0.0070 - accuracy: 0.9976

Test accuracy: 0.9975860118865967
1360/1360 - 2s - loss: 0.0067 - accuracy: 0.9974
Epoch 1/10
1360/1360 [==============================] - 3s 2ms/step - loss: 1.3933 - accuracy: 0.7701
Epoch 2/10
1360/1360 [==============================] - 3s 2ms/step - loss: 0.4798 - accuracy: 0.9694
Epoch 3/10
1360/1360 [==============================] - 3s 2ms/

In [37]:
sum(sf)/30

0.48392427757334383

In [39]:
sum(sta)/30

0.9981195012728373

In [40]:
sum(ste)/30

0.998305634657542

In [41]:
max(sf)

0.8974358974358975

In [42]:
max(ste)

0.9994482398033142

In [43]:
max(sta)

0.9994482398033142

In [44]:
stdev(sf)

0.4203400942606391

In [45]:
stdev(ste)

0.0010545292968867001

In [46]:
stdev(sta)

0.0012293893176812752

NEURAL NETWORK WITH SMOTE:

In [47]:
sta2=[]
ste2=[]
sf2 =[]
for i in range(30):
    r=callf1(X_train_res,y_train_res.ravel(),Xtes,ytest.ravel())
    ste2.append(r[0])
    sta2.append(r[1])
    sf2.append(r[2])

Epoch 1/10
2711/2711 [==============================] - 7s 2ms/step - loss: 0.7810 - accuracy: 0.8627
Epoch 2/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.1539 - accuracy: 0.9962
Epoch 3/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0681 - accuracy: 0.9979
Epoch 4/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0338 - accuracy: 0.9984
Epoch 5/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0210 - accuracy: 0.9986
Epoch 6/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0122 - accuracy: 0.9991
Epoch 7/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0082 - accuracy: 0.9992
Epoch 8/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0066 - accuracy: 0.9992
Epoch 9/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0055 - accuracy: 0.9993
Epoch 10/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.005

2711/2711 [==============================] - 6s 2ms/step - loss: 0.1438 - accuracy: 0.9968
Epoch 3/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0699 - accuracy: 0.9985
Epoch 4/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0250 - accuracy: 0.9986
Epoch 5/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0143 - accuracy: 0.9989
Epoch 6/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0104 - accuracy: 0.9991
Epoch 7/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0092 - accuracy: 0.9991
Epoch 8/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0077 - accuracy: 0.9994
Epoch 9/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0080 - accuracy: 0.9992
Epoch 10/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0078 - accuracy: 0.9992
454/454 - 1s - loss: 0.0075 - accuracy: 0.9986

Test accuracy: 0.9986205697059631
2711/2711 

2711/2711 [==============================] - 7s 2ms/step - loss: 0.0570 - accuracy: 0.9982
Epoch 4/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0189 - accuracy: 0.9988
Epoch 5/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0121 - accuracy: 0.9992
Epoch 6/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0076 - accuracy: 0.9993
Epoch 7/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0061 - accuracy: 0.9995
Epoch 8/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0056 - accuracy: 0.9995
Epoch 9/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0050 - accuracy: 0.9996
Epoch 10/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0050 - accuracy: 0.9996
454/454 - 1s - loss: 0.0016 - accuracy: 0.9991

Test accuracy: 0.9991033673286438
2711/2711 - 3s - loss: 0.0042 - accuracy: 0.9996
Epoch 1/10
2711/2711 [==============================] - 7s 2ms/

2711/2711 [==============================] - 6s 2ms/step - loss: 0.0225 - accuracy: 0.9990
Epoch 5/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0108 - accuracy: 0.9994
Epoch 6/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0067 - accuracy: 0.9994
Epoch 7/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0057 - accuracy: 0.9993
Epoch 8/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0048 - accuracy: 0.9995
Epoch 9/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0046 - accuracy: 0.9996
Epoch 10/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0046 - accuracy: 0.9994
454/454 - 1s - loss: 0.0010 - accuracy: 0.9990

Test accuracy: 0.9990344047546387
2711/2711 - 3s - loss: 0.0039 - accuracy: 0.9996
Epoch 1/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.7129 - accuracy: 0.8889
Epoch 2/10
2711/2711 [==============================] - 6s 2ms/

2711/2711 [==============================] - 6s 2ms/step - loss: 0.0157 - accuracy: 0.9991
Epoch 6/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0077 - accuracy: 0.9994
Epoch 7/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0065 - accuracy: 0.9993
Epoch 8/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0053 - accuracy: 0.9994
Epoch 9/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0047 - accuracy: 0.9995
Epoch 10/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0042 - accuracy: 0.9995
454/454 - 1s - loss: 0.0012 - accuracy: 0.9989

Test accuracy: 0.9988964796066284
2711/2711 - 3s - loss: 0.0028 - accuracy: 0.9996
Epoch 1/10
2711/2711 [==============================] - 7s 2ms/step - loss: 0.7260 - accuracy: 0.8396
Epoch 2/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.1496 - accuracy: 0.9964
Epoch 3/10
2711/2711 [==============================] - 6s 2ms/

In [53]:
sum(sf2)/30

0.8079373240587466

In [55]:
sum(ste2)/30

0.9987424433231353

In [56]:
sum(sta2)/30

0.9994754056135814

In [57]:
max(sf2)

0.853932584269663

In [58]:
max(ste2)

0.9991033673286438

In [59]:
max(sta2)

0.9995964765548706

In [60]:
stdev(sf2)

0.028931551520662145

In [61]:
stdev(ste2)

0.00023593470168102089

In [62]:
stdev(sta2)

7.699825178096488e-05

In [63]:
X.shape

(43499, 9)

In [26]:
X_oversampled=X_train_res[43499:]

In [27]:
X_oversampled.shape

(43235, 9)

GAN MODEL:

In [65]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [66]:
torch.cuda.empty_cache()

In [67]:
def get_generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True),
    )

In [68]:
class Generator(nn.Module):

    def __init__(self, z_dim=9, im_dim=9, hidden_dim=128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            get_generator_block(z_dim, hidden_dim),
            get_generator_block(hidden_dim, hidden_dim * 2),
            get_generator_block(hidden_dim * 2, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            nn.Linear(hidden_dim * 8, im_dim),
            nn.Sigmoid()
        )
    def forward(self, noise):
        return self.gen(noise)
    
    # Needed for grading
    def get_gen(self):

        return self.gen

In [69]:
def get_discriminator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.LeakyReLU(0.2, inplace=True)        
    )

In [70]:
class Discriminator(nn.Module):
    def __init__(self, im_dim=9, hidden_dim=128):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim * 4),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            get_discriminator_block(hidden_dim * 2, hidden_dim),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, image):

        return self.disc(image)
    
    def get_disc(self):

        return self.dis

In [71]:
X_oversampled = torch.from_numpy(X_oversampled)

TypeError: expected np.ndarray (got Tensor)

In [72]:
criterion = nn.BCEWithLogitsLoss()
n_epochs = 400
z_dim = 9
batch_size = 128
lr = 0.00001
display_step = 500
device = 'cuda'

In [73]:
gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)
disc = Discriminator().to(device) 
disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)

In [74]:
def get_disc_loss(gen, disc, criterion, real, device):

    fake = gen(X_oversampled.float().to(device))
    disc_fake_pred = disc(fake.detach())
    disc_fake_loss = criterion(disc_fake_pred, torch.zeros_like(disc_fake_pred))
    disc_real_pred = disc(real)
    disc_real_loss = criterion(disc_real_pred, torch.ones_like(disc_real_pred))
    disc_loss = (disc_fake_loss + disc_real_loss) / 2
    return disc_loss

In [75]:
def get_gen_loss(gen, disc, criterion, num_images, z_dim, device):
    fake_images = gen(X_oversampled.float().to(device))
    
    disc_fake_pred = disc(fake_images)
    gen_loss = criterion(disc_fake_pred, torch.ones_like(disc_fake_pred))
    return gen_loss

In [76]:
y_tr=ytrain.ravel()

In [77]:
li=[]
for i in range(len(y_tr)):
    if int(y_tr[i])==1:
        li.append(X[i])

In [78]:
X_real=np.array(li)

In [79]:
X_real.shape

(132, 9)

In [80]:
li2=[1]*132

In [81]:
y_real=np.array(li2)
y_real.shape

(132,)

In [82]:
from torch.utils.data import TensorDataset, DataLoader
tensor_x = torch.Tensor(X_real) 
tensor_y = torch.Tensor(y_real)
my_dataset = TensorDataset(tensor_x,tensor_y) # create your datset

In [83]:
dataloader = DataLoader(
    my_dataset,
    batch_size=batch_size,
    shuffle=True)

In [84]:

cur_step = 0
mean_generator_loss = 0
mean_discriminator_loss = 0
test_generator = True 
gen_loss = False
error = False
for epoch in range(n_epochs):
  
    # Dataloader returns the batches
    for real, _ in tqdm(dataloader):
        cur_batch_size = len(real)

        # Flatten the batch of real images from the dataset
        real = real.view(cur_batch_size, -1).to(device)

        ### Update discriminator ###
        # Zero out the gradients before backpropagation
        disc_opt.zero_grad()

        # Calculate discriminator loss
        disc_loss = get_disc_loss(gen, disc, criterion, real, device)

        # Update gradients#
        disc_loss.backward(retain_graph=True)

        # Update optimizer
        disc_opt.step()

        # For testing purposes, to keep track of the generator weights
        if test_generator:
            old_generator_weights = gen.gen[0][0].weight.detach().clone()

        ### Update generator ###
        #     Hint: This code will look a lot like the discriminator updates!
        #     These are the steps you will need to complete:
        #       1) Zero out the gradients.
        #       2) Calculate the generator loss, assigning it to gen_loss.
        #       3) Backprop through the generator: update the gradients and optimizer.
        #### START CODE HERE ####
        gen_opt.zero_grad()
        gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device)
        gen_loss.backward()
        gen_opt.step()        
        #### END CODE HERE ####

        # For testing purposes, to check that your code changes the generator weights
        if test_generator:
            try:
                assert lr > 0.0000002 or (gen.gen[0][0].weight.grad.abs().max() < 0.0005 and epoch == 0)
                assert torch.any(gen.gen[0][0].weight.detach().clone() != old_generator_weights)
            except:
                error = True
                print("Runtime tests have failed")

        # Keep track of the average discriminator loss
        mean_discriminator_loss += disc_loss.item() / display_step

        # Keep track of the average generator loss
        mean_generator_loss += gen_loss.item() / display_step

        ### Visualization code ###
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Epoch {epoch}, step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
            #fake_noise = get_noise(cur_batch_size, z_dim, device=device)
            #fake = gen(fake_noise)
            #show_tensor_images(fake)
            #show_tensor_images(real)
            mean_generator_loss = 0
            mean_discriminator_loss = 0
        cur_step += 1


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [85]:
res=gen(X_oversampled.float().to(device))

In [86]:
res

tensor([[0.0369, 0.0125, 0.0877,  ..., 0.0265, 0.0284, 0.1842],
        [0.0392, 0.0094, 0.9682,  ..., 0.1510, 0.0722, 0.0221],
        [0.0301, 0.0105, 0.0665,  ..., 0.0187, 0.0238, 0.2257],
        ...,
        [0.0402, 0.0151, 0.8077,  ..., 0.1163, 0.0733, 0.0264],
        [0.0285, 0.0148, 0.1109,  ..., 0.0227, 0.0365, 0.0361],
        [0.0329, 0.0122, 0.0730,  ..., 0.0246, 0.0253, 0.1290]],
       device='cuda:0', grad_fn=<SigmoidBackward>)

In [87]:
X_oversampled

tensor([[  45.9456,  -32.7825,   89.0000,  ...,   43.0544,   84.0000,
           41.8913],
        [  85.3713,  -36.3713,  107.0000,  ...,   21.3144,   42.0000,
           21.3713],
        [  37.1115,  -41.7323,   76.0000,  ...,   38.8885,   39.8885,
            2.0000],
        ...,
        [  74.4796,  -42.7195,  107.0000,  ...,   32.2805,   45.0000,
           12.4796],
        [  37.0000, -113.4671,  106.0000,  ...,   69.0000,   74.6421,
            5.7614],
        [  46.3826,  -48.6174,   77.0000,  ...,   30.2347,   79.0000,
           48.7653]])

In [89]:
fres=res.cpu().detach().numpy()
fres.shape

(43235, 9)

NEURAL NETWORK SMOTE+GAN DATA:

In [94]:
fin=np.concatenate((X_train_res[:43499], fres), axis=0)

In [95]:
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [96]:
Xn,yn=shuffle_in_unison(fin, y_train_res)


In [97]:
Xn.shape

(86734, 9)

In [169]:
sta3=[]
ste3=[]
sf3 =[]
for i in range(30):
    r=callf1(Xn,yn.ravel(),Xtes,ytest.ravel())
    ste3.append(r[0])
    sta3.append(r[1])
    sf3.append(r[2])

Epoch 1/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.7779 - accuracy: 0.8892
Epoch 2/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.1246 - accuracy: 0.9974
Epoch 3/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0405 - accuracy: 0.9981
Epoch 4/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0187 - accuracy: 0.9980
Epoch 5/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0060 - accuracy: 0.9983
Epoch 6/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0029 - accuracy: 0.9984
Epoch 7/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0030 - accuracy: 0.9984
Epoch 8/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0029 - accuracy: 0.9984
Epoch 9/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0025 - accuracy: 0.9987
Epoch 10/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.002

2711/2711 [==============================] - 5s 2ms/step - loss: 0.0993 - accuracy: 0.9975
Epoch 3/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0196 - accuracy: 0.9977
Epoch 4/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0038 - accuracy: 0.9984
Epoch 5/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0030 - accuracy: 0.9990
Epoch 6/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0025 - accuracy: 0.9993
Epoch 7/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0021 - accuracy: 0.9996
Epoch 8/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0022 - accuracy: 0.9993
Epoch 9/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0023 - accuracy: 0.9994
Epoch 10/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0022 - accuracy: 0.9994
454/454 - 1s - loss: 0.0017 - accuracy: 0.9992

Test accuracy: 0.9992413520812988
2711/2711 

2711/2711 [==============================] - 5s 2ms/step - loss: 0.0399 - accuracy: 0.9978
Epoch 4/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0145 - accuracy: 0.9982
Epoch 5/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0073 - accuracy: 0.9981
Epoch 6/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0033 - accuracy: 0.9988
Epoch 7/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0022 - accuracy: 0.9996
Epoch 8/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0024 - accuracy: 0.9994
Epoch 9/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0021 - accuracy: 0.9996
Epoch 10/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0022 - accuracy: 0.9995
454/454 - 1s - loss: 0.0010 - accuracy: 0.9994

Test accuracy: 0.9993792772293091
2711/2711 - 4s - loss: 0.0014 - accuracy: 0.9997
Epoch 1/10
2711/2711 [==============================] - 5s 2ms/

2711/2711 [==============================] - 5s 2ms/step - loss: 0.0062 - accuracy: 0.9982
Epoch 5/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0033 - accuracy: 0.9984
Epoch 6/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0030 - accuracy: 0.9987
Epoch 7/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0027 - accuracy: 0.9992
Epoch 8/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0023 - accuracy: 0.9995
Epoch 9/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0023 - accuracy: 0.9993
Epoch 10/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0020 - accuracy: 0.9995
454/454 - 1s - loss: 0.0016 - accuracy: 0.9988

Test accuracy: 0.9988275170326233
2711/2711 - 3s - loss: 0.0031 - accuracy: 0.9994
Epoch 1/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.9028 - accuracy: 0.8830
Epoch 2/10
2711/2711 [==============================] - 5s 2ms/

2711/2711 [==============================] - 5s 2ms/step - loss: 0.0028 - accuracy: 0.9991
Epoch 6/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0025 - accuracy: 0.9995
Epoch 7/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0027 - accuracy: 0.9994
Epoch 8/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0024 - accuracy: 0.9994
Epoch 9/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0023 - accuracy: 0.9995
Epoch 10/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.0023 - accuracy: 0.9994
454/454 - 1s - loss: 9.7717e-04 - accuracy: 0.9994

Test accuracy: 0.9993792772293091
2711/2711 - 3s - loss: 0.0017 - accuracy: 0.9997
Epoch 1/10
2711/2711 [==============================] - 6s 2ms/step - loss: 0.6615 - accuracy: 0.8852
Epoch 2/10
2711/2711 [==============================] - 5s 2ms/step - loss: 0.0564 - accuracy: 0.9982
Epoch 3/10
2711/2711 [==============================] - 5s 

In [210]:
sum(sf3)/30

0.8224166277591309

In [211]:
sum(sta3)/30

0.9994158426920573

In [212]:
sum(ste3)/30

0.9988413016001384

In [213]:
max(sf3)

0.9135802469135802

In [214]:
max(ste3)

0.9995172023773193

In [215]:
max(sta3)

0.9997809529304504

In [216]:
stdev(sf3)

0.0719860408419574

In [217]:
stdev(sta3)

0.00040225428258809225

In [218]:
stdev(ste3)

0.0006727021942221619

In [219]:
save=1


In [221]:
progressnew1=1